In [1]:
import numpy as np
import random
import pickle
from simple_custom_taxi_env import SimpleTaxiEnv, run_agent
import time

In [6]:
np.bool8 = np.bool_

env_config = {
    "fuel_limit": 5000
}
render = False
hyperparameters = {
    "alpha": 0.01,
	"gamma": 0.99,
	"epsilon_start": 1.0, 
	"epsilon_end": 0.01,
	"decay_rate": 0.9999,
	"episodes": 25000,
 	"max_steps": 2000
}

In [8]:
q_table = {}
epsilon = hyperparameters["epsilon_start"]

def random_action(passenger_look, destination_look, has_picked_up, obs):
	action_probs = np.ones(action_nums) / action_nums
	if obstacle_south:
		action_probs[0] = 0
	if obstacle_north:
		action_probs[1] = 0
	if obstacle_east:
		action_probs[2] = 0
	if obstacle_west:
		action_probs[3] = 0	
	if not passenger_look or has_picked_up or not is_in_station(obs):
		action_probs[4] = 0
	if not destination_look or not has_picked_up or not is_in_station(obs):
		action_probs[5] = 0
	if np.sum(action_probs) == 0:
		action_probs = np.ones(action_nums) / action_nums
	action_probs = action_probs / np.sum(action_probs)
	action = np.random.choice(action_nums, p=action_probs)  # Random action
	return action

def get_state(obs, target_loc=None, has_picked_up=False):
	stations = [[0, 0], [0, 4], [4, 0], [4,4]]
	taxi_row, taxi_col, stations[0][0],stations[0][1] ,stations[1][0],stations[1][1],stations[2][0],stations[2][1],stations[3][0],stations[3][1],obstacle_north, obstacle_south, obstacle_east, obstacle_west, passenger_look, destination_look = obs
	stations = [tuple(i) for i in stations]	

	assert target_loc is not None
	x_dir = target_loc[0] - taxi_row
	y_dir = target_loc[1] - taxi_col
	x_dir = 0 if x_dir == 0 else x_dir // abs(x_dir)
	y_dir = 0 if y_dir == 0 else y_dir // abs(y_dir)
	return (x_dir, y_dir, obstacle_north, obstacle_south, obstacle_east, obstacle_west, passenger_look, destination_look, has_picked_up)
	# return (obstacle_north, obstacle_south, obstacle_east, obstacle_west)
  
def get_action(obs):
	"""
	# Selects the best action using the trained Q-table.
	"""
	if np.random.uniform(0, 1) < epsilon:
		action = np.random.choice(action_nums)  # Random action
	else:
		action = np.argmax(q_table[get_state(obs)])  # Greedy action
	return action

def is_in_station(obs):
	"""
	# Checks if the taxi is in a station.
	"""
	stations = [[0, 0], [0, 4], [4, 0], [4,4]]
	taxi_row, taxi_col,stations[0][0],stations[0][1] ,stations[1][0],stations[1][1],stations[2][0],stations[2][1],stations[3][0],stations[3][1],obstacle_north, obstacle_south, obstacle_east, obstacle_west, passenger_look, destination_look = obs
	stations = [tuple(i) for i in stations]
	return (taxi_row, taxi_col) in stations

env = SimpleTaxiEnv(**env_config)
action_nums = 6
rewards_per_episode = []

obs, _ = env.reset()
total_reward = 0
done = False
step_count = 0
stations = [(0, 0), (0, 4), (4, 0), (4,4)]

taxi_row, taxi_col, _,_,_,_,_,_,_,_,obstacle_north, obstacle_south, obstacle_east, obstacle_west, passenger_look, destination_look = obs

if render:
	env.render_env((taxi_row, taxi_col),
					action=None, step=step_count, fuel=env.current_fuel)
	time.sleep(0.5)
 

achieved = []
step_counts = []
for episode in range(hyperparameters["episodes"]):
	get_state.passenger_loc, get_state.destination_loc = None, None
	env = SimpleTaxiEnv(grid_size=np.random.choice([5, 6, 7, 8, 9]), **env_config)
	obs, _ = env.reset()
	done = False
	total_reward = 0
	step_count = 0
 
	destination = None
	visited = []
	has_picked_up = False
	violations = []
 
	taxi_row, taxi_col, _,_,_,_,_,_,_,_,obstacle_north, obstacle_south, obstacle_east, obstacle_west, passenger_look, destination_look = obs
	stations = [(obs[2], obs[3]), (obs[4], obs[5]), (obs[6], obs[7]), (obs[8], obs[9])]
	target_loc = random.choice(stations)
	state = get_state(obs, target_loc, has_picked_up)
	
	while not done:	
		if state not in q_table:
			q_table[state] = np.zeros(action_nums)
   
		taxi_row, taxi_col, _,_,_,_,_,_,_,_,obstacle_north, obstacle_south, obstacle_east, obstacle_west, passenger_look, destination_look = obs
		if np.random.uniform(0, 1) < epsilon:
			action = random_action(passenger_look, destination_look, has_picked_up, obs)
		else:
			action = np.argmax(q_table[state])  # Greedy action
		
		shaped_reward = 0
   
		x_dir = target_loc[0] - taxi_row
		y_dir = target_loc[1] - taxi_col
		x_dir = 0 if x_dir == 0 else x_dir // abs(x_dir)
		y_dir = 0 if y_dir == 0 else y_dir // abs(y_dir)
		# if action == 0 :  # Move Down
        #     next_row += 1
        # elif action == 1:  # Move Up
        #     next_row -= 1
        # elif action == 2:  # Move Right
        #     next_col += 1
        # elif action == 3:  # Move Left
        #     next_col -= 1
		# if y_dir == 1 and action == 2 and not obstacle_east:
		# 	shaped_reward = 10
		# if y_dir == -1 and action == 3 and not obstacle_west:
		# 	shaped_reward = 10
		# if x_dir == 1 and action == 0 and not obstacle_south:
		# 	# print (target_loc, (taxi_row, taxi_col), (x_dir, y_dir), action)
		# 	shaped_reward = 10
		# if x_dir == -1 and action == 1 and not obstacle_north:
		# 	shaped_reward = 10
		if action == 4 and passenger_look and not has_picked_up and is_in_station(obs): 
			shaped_reward += 100
			has_picked_up = True
		if action == 5 and destination_look and has_picked_up and is_in_station(obs):
			shaped_reward += 100
			done = True
   
		if obstacle_south and action == 0:
			violations.append(0)
			shaped_reward += -20
			# print ("south")
			# print ((taxi_row, taxi_col), (x_dir, y_dir), obstacle_south)
		if obstacle_north and action == 1:
			violations.append(1)
			shaped_reward += -100
			# print ("north")
		if obstacle_east and action == 2:
			violations.append(2)
			shaped_reward += -100
			# print ("east")
		if obstacle_west and action == 3:
			violations.append(3)
			shaped_reward += -100
			# print ("west")
		if (not passenger_look or has_picked_up or not is_in_station(obs)) and action == 4:
			violations.append(4)
			shaped_reward += -100
			# print ("not passenger")
		if (not destination_look or not has_picked_up or not is_in_station(obs)) and action == 5:
			violations.append(5)
			shaped_reward += -100
			# print ("not destination")

		prev_dist = abs(taxi_row - target_loc[0]) + abs(taxi_col - target_loc[1])
  
		next_obs, reward, done, _ = env.step(action)
		taxi_row, taxi_col, _,_,_,_,_,_,_,_,obstacle_north, obstacle_south, obstacle_east, obstacle_west, passenger_look, destination_look = next_obs
  
		# next_dist = abs(taxi_row - target_loc[0]) + abs(taxi_col - target_loc[1])
		# if next_dist < prev_dist:
		# 	shaped_reward += 10
  
		if is_in_station(next_obs):
			visited.append((taxi_row, taxi_col))
			if destination_look:
				destination = (taxi_row, taxi_col)
			if has_picked_up and destination is not None:
				target_loc = destination
			else:
				# choose a station that has not been visited yet
				# print (visited, (taxi_row, taxi_col))
				for station in stations:
					if station not in visited:
						target_loc = station
						break	
				# print (target_loc)
		# print (destination, (taxi_row, taxi_col), action, shaped_reward)
		if done:
			if step_count == 4999:
				achieved.append(0)
			elif step_count < 4999:
				achieved.append(1)
				shaped_reward += 200
			step_counts.append(step_count)
		# if done:
		# 	if not has_picked_up:
		# 		# shaped_reward = -100
		# 		# print ("not picked up", next_obs, action, has_picked_up, destination)
		# 		done = False
		# 	if has_picked_up and not destination_look:
		# 		# shaped_reward = -100
		# 		# print ("not destination")
		# 		done = False
		# 	if has_picked_up and destination_look:
		# 		# print ("done")
		# 		shaped_reward = 100
		# 		step_counts.append(step_count)
   
		total_reward += reward
		reward += shaped_reward
		next_state = get_state(next_obs, target_loc, has_picked_up)
  
		if next_state not in q_table:
			q_table[next_state] = np.zeros(action_nums)
   
		q_table[state][action] += hyperparameters["alpha"] * (reward + hyperparameters["gamma"] * np.max(q_table[next_state]) - q_table[state][action])
		
		step_count += 1
		obs = next_obs
		state = next_state
		
		if render:
			env.render_env((taxi_row, taxi_col),
							action=action, step=step_count, fuel=env.current_fuel)
	# print (np.sum(np.array(violations) == 0), np.sum(np.array(violations) == 1), np.sum(np.array(violations) == 2), np.sum(np.array(violations) == 3), np.sum(np.array(violations) == 4), np.sum(np.array(violations) == 5))
	rewards_per_episode.append(total_reward)
	epsilon = max(hyperparameters["epsilon_end"], epsilon * hyperparameters["decay_rate"])
	if (episode + 1) % 100 == 0:
		avg_reward = np.mean(rewards_per_episode[-100:])
		print(f'Episode {episode + 1}/{hyperparameters["episodes"]}, Avg Reward: {avg_reward:.4f}, Epsilon: {epsilon:.3f}, Achieved: {np.mean(achieved[-100:]):.2f}, Avg Steps: {np.mean(step_counts[-100:]):.2f}')
		# print ([np.argmax(i) for i in q_table.values()])
		print (len(q_table))
  

Episode 100/25000, Avg Reward: -341.8550, Epsilon: 0.990, Achieved: 0.48, Avg Steps: 3008.05
491
Episode 200/25000, Avg Reward: -601.6150, Epsilon: 0.980, Achieved: 0.60, Avg Steps: 2522.65
548
Episode 300/25000, Avg Reward: -165.0830, Epsilon: 0.970, Achieved: 0.74, Avg Steps: 1860.33
564
Episode 400/25000, Avg Reward: -159.4010, Epsilon: 0.961, Achieved: 0.73, Avg Steps: 1844.51
573
Episode 500/25000, Avg Reward: -171.1420, Epsilon: 0.951, Achieved: 0.70, Avg Steps: 1958.92
582
Episode 600/25000, Avg Reward: -179.2250, Epsilon: 0.942, Achieved: 0.71, Avg Steps: 2037.75
583
Episode 700/25000, Avg Reward: -161.5350, Epsilon: 0.932, Achieved: 0.72, Avg Steps: 1840.35
584
Episode 800/25000, Avg Reward: -202.2470, Epsilon: 0.923, Achieved: 0.64, Avg Steps: 2171.97
586
Episode 900/25000, Avg Reward: -503.5340, Epsilon: 0.914, Achieved: 0.67, Avg Steps: 2004.34
593
Episode 1000/25000, Avg Reward: -192.4550, Epsilon: 0.905, Achieved: 0.67, Avg Steps: 2093.05
594
Episode 1100/25000, Avg Rewar

KeyboardInterrupt: 

In [9]:
filename = "q_table_1.pkl"

# 儲存 Q-table
with open(filename, "wb") as f:
    pickle.dump(q_table, f)

print(f"Q-table 已儲存至 {filename}")

Q-table 已儲存至 q_table_1.pkl


In [11]:

env_config = {
        "grid_size": np.random.randint(5, 10),
        "fuel_limit": 5000
    }
    
agent_score = run_agent("student_agent.py", env_config, render=False)
print(f"Final Score: {agent_score}")

613
[0, 0]
(-1, -1, 0, 0, 0, 0, 0, 0, False) 3
obs= (5, 2, 0, 0, 0, 7, 7, 0, 7, 7, 0, 0, 0, 1, 0, 0)
(-1, -1, 0, 0, 0, 1, 0, 0, False) 1
obs= (4, 2, 0, 0, 0, 7, 7, 0, 7, 7, 0, 0, 0, 0, 0, 0)
(-1, -1, 0, 0, 0, 0, 0, 0, False) 3
obs= (4, 1, 0, 0, 0, 7, 7, 0, 7, 7, 0, 1, 0, 0, 0, 0)
(-1, -1, 0, 1, 0, 0, 0, 0, False) 3
obs= (4, 0, 0, 0, 0, 7, 7, 0, 7, 7, 0, 0, 0, 1, 0, 0)
(-1, 0, 0, 0, 0, 1, 0, 0, False) 0
obs= (5, 0, 0, 0, 0, 7, 7, 0, 7, 7, 0, 0, 1, 1, 0, 0)
(-1, 0, 0, 0, 1, 1, 0, 0, False) 1
obs= (4, 0, 0, 0, 0, 7, 7, 0, 7, 7, 0, 0, 0, 1, 0, 0)
(-1, 0, 0, 0, 0, 1, 0, 0, False) 0
obs= (5, 0, 0, 0, 0, 7, 7, 0, 7, 7, 0, 0, 1, 1, 0, 0)
(-1, 0, 0, 0, 1, 1, 0, 0, False) 1
obs= (4, 0, 0, 0, 0, 7, 7, 0, 7, 7, 0, 0, 0, 1, 0, 0)
(-1, 0, 0, 0, 0, 1, 0, 0, False) 0
obs= (5, 0, 0, 0, 0, 7, 7, 0, 7, 7, 0, 0, 1, 1, 0, 0)
(-1, 0, 0, 0, 1, 1, 0, 0, False) 1
obs= (4, 0, 0, 0, 0, 7, 7, 0, 7, 7, 0, 0, 0, 1, 0, 0)
(-1, 0, 0, 0, 0, 1, 0, 0, False) 0
obs= (5, 0, 0, 0, 0, 7, 7, 0, 7, 7, 0, 0, 1, 1, 0, 0)
(-1, 0